In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS, StaticAssets
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline.data import morningstar, Fundamentals
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage



def make_pipeline(ticker):
    """
    A function to create our dynamic stock selector (pipeline). Documentation
    on pipeline can be found here:
    https://www.quantopian.com/help#pipeline-title
    """

    # Base universe set to the QTradableStocksUS
    mask = QTradableStocksUS() & Sector().notnull()
    mask &= (Fundamentals.market_cap.latest > 1000000000)
    mask &= (
        (Fundamentals.morningstar_sector_code.latest != 103) &
        (Fundamentals.morningstar_sector_code.latest != 207) &
        (Fundamentals.morningstar_sector_code.latest != 206) &
        (Fundamentals.morningstar_sector_code.latest != 309) &
        (Fundamentals.morningstar_industry_code.latest != 20533080) &
        (Fundamentals.morningstar_industry_code.latest != 10217033) &
        (Fundamentals.morningstar_industry_group_code != 10106) &
        (Fundamentals.morningstar_industry_group_code != 10104)
    )
    #mask = StaticAssets(symbols(['GOOG', 'AMZN', 'MSFT', 'AAPL', 'DIS', 'TXN', 'SCHW']))
    #mask = StaticAssets(symbols(['SPY']))

    mask = StaticAssets(symbols([ticker]))
    
    # Factor of yesterday's close price.
    yesterday_close = USEquityPricing.close.latest
    yesterday_open = USEquityPricing.open.latest

    # Uncomment to include value and rank metric pairs in MF calculation
    columns = {
            'close' : yesterday_close,
            'open'  : yesterday_open,
            'change': (yesterday_close - yesterday_open) / yesterday_open * 100,
        }
    
    pipe = Pipeline(
        columns=columns,
        screen=mask
    )
    return pipe


ticker = 'spy'

# 2008 crisis
# high 2007-10-08
# low 2009-03-02
#result = run_pipeline(make_pipeline(ticker), '2007-10-08', '2009-03-02')
# recovery
#result = run_pipeline(make_pipeline(ticker), '2009-03-02', '2011-03-02')
# full
result = run_pipeline(make_pipeline(ticker), '2007-10-08', '2010-03-02')

# bull run
#result = run_pipeline(make_pipeline(ticker), '2014-03-02', '2018-03-02')

bins = [-float('inf'), -3, -2, -1, 0, 1, 2, 3, float('inf')]
plt.hist(result['change'], bins=20)

print(ticker)
print('above 0', sum(result['change'] > 0))
print('below 0', sum(result['change'] < 0))
print('up', sum(result['change'] > 0)/len(result['change']))


#bounds = pd.cut(result['change'], bins)
#counts = bounds.value_counts()
#plt.bar(counts.values, counts.index)



